## CPAL versus Deep Active Learning Baselines on Synthetic Spiral

In this tutorial, we walk through how to use CPAL (with two-layer ReLU Networks), linear cutting-plane active learning, DeepAL baselines, and Skactive baselines on a binary classification task using a synthetic spiral.

In [ ]:
import sys
sys.path.append('..')
from src.cpal.cpal import *
from src.baselines.linear_cp import *
from types import SimpleNamespace
from deepal_baseline.deepal_class import run_active_learning
from src.baselines.evaluation import *
from src.cpal.synthetic_data import *
from src.cpal.plot import *

### (1). Generate synthetic data

In [ ]:
X_all, y_all, X, y, X_test, y_test = generate_spiral_data()

### (2). Sample hyperplane arrangements

In [ ]:
dmat = generate_hyperplane_arrangement(X = X, P = 1000)
n_train, m = dmat.shape
print(f'dmat shape: {dmat.shape}')

### (3). CPAL with two-layer ReLU networks (Before-Final-Solve vs. After-Final-Solve)

In [ ]:
C, c, used = cutting_plane('c',X, y, dmat, n_points = 20)
print(f'used: {used}')

In [ ]:
n_train, m = dmat.shape
d = X.shape[1]
theta_matrix = np.reshape(c, (2*d,m), order ='F') # the resulting theta_matrix is of shape (2d, m)
Uopt1_v = theta_matrix[:d]
Uopt2_v = theta_matrix[d:]
# do the final convex solve
Uopt1_final_v, Uopt2_final_v, _ = convex_solve(used, X, y, dmat)

In [ ]:
print('------------')
print('Model evaluation for CPAL (before final convex solve).')
evaluate_model_performance(task = 'c', X_test = X_test, y_test = y_test, Uopt1 = Uopt1_v, Uopt2 = Uopt2_v, X_train = X, y_train = y)

In [ ]:
print('------------')
print('Model evaluation for CPAL (after final convex solve).')
evaluate_model_performance(task = 'c', X_test = X_test, y_test = y_test, Uopt1 = Uopt1_final_v, Uopt2 = Uopt2_final_v, X_train = X, y_train = y)

In [ ]:
# plot decision boundary for both
plot_decision_boundary(
    X=X,
    y=y,
    X_test=X_test,
    y_test=y_test,
    name='Decision Boundary for CPAL (Before final solve)',
    selected_indices = used,
    Uopt1v=Uopt1_v,
    Uopt2v=Uopt2_v,
    plot_type='cvx'
)

In [ ]:
plot_decision_boundary(
    X=X,
    y=y,
    X_test=X_test,
    y_test=y_test,
    selected_indices=used,
    name='Decision Boundary for CPAL (After final solve)',
    Uopt1v=Uopt1_final_v,
    Uopt2v=Uopt2_final_v,
    plot_type='cvx'
)

### (4). Linear Cutting-Plane Baseline

In [ ]:
C, c, used = linear_cutting_plane_classification(X, y, m, 20)
print(f'used: {used}')

In [ ]:
plot_decision_boundary_linear(X_all, X, y, X_test, y_test, c, used)

### (5). Run DeepAL baselines

Try our sample runs by calling the following commands in the terminal:

``python -m deepal_baseline.demo \
   --n_round 3 \
   --n_query 10 \
   --n_init_labeled 10 \
   --dataset_name Spiral \
   --strategy_name EntropySampling\
   --seed 1``

   Or equivalently, use:

In [ ]:
deepal_selected_indices = []
deepal_strategies = ["RandomSampling", "LeastConfidence", "EntropySampling","KMeansSampling", "BALDDropout"]


for al in deepal_strategies:
    args = SimpleNamespace(
        seed=1,
        n_init_labeled=1,
        n_query=1,
        n_round=14,
        dataset_name='Spiral',
        strategy_name=al,
        save_dir="class_plots",
        save_pickle="class_accuracy"
    )

    selected_indices, rmse_train, rmse_test, strategy, dataset, net, device = run_active_learning(args)
    deepal_selected_indices.append(selected_indices)